O parâmetro **`on_disk`** é, talvez, a configuração mais importante para viabilizar projetos de IA que lidam com milhões de dados sem gastar uma fortuna com servidores.

Para entender de verdade, imagine a diferença entre a **Memória RAM** e o **Disco (SSD)** do seu computador:

* **RAM:** É ultrarrápida, mas pequena e cara.
* **Disco:** É mais lento que a RAM, mas é gigantesco e muito barato.

---

### A Analogia da "Mesa de Trabalho" vs. "Arquivo de Aço"

Imagine que você é um bibliotecário processando pedidos de busca:

1. **`on_disk: False` (Padrão):** Você mantém todos os livros abertos em cima da sua mesa (RAM). Quando alguém pergunta algo, você olha para a mesa e responde instantaneamente. **Problema:** Se chegarem 1 milhão de livros, sua mesa não terá espaço e você precisará comprar uma mesa (servidor) gigante e caríssima.
2. **`on_disk: True`:** Você guarda os livros em grandes arquivos de aço (Disco/SSD). Na sua mesa, você mantém apenas um "índice" ou um resumo colado (Quantização). Quando alguém faz uma pergunta, você usa o resumo para saber quais arquivos abrir, vai até o armário, pega o livro e lê o detalhe.

### O que acontece tecnicamente?

Quando você define `on_disk: True` na configuração dos vetores, o Qdrant utiliza uma tecnologia chamada **Memory Mapping (mmap)**.

* **Sem `on_disk`:** O Qdrant carrega todos os vetores (as listas de números decimais) para a memória RAM. Se você tem 1 milhão de vetores de 1536 dimensões (padrão da OpenAI), isso ocuparia aproximadamente  GB de RAM apenas para os vetores brutos.
* **Com `on_disk`:** Os vetores ficam armazenados no seu SSD. O sistema operacional "mapeia" esses arquivos. O Qdrant só carrega o vetor específico para a RAM no exato momento em que precisa calcular a distância final ou retornar o resultado para você.

### Por que isso é vital para o "Day 4" do curso?

No curso, você aprende sobre o dataset **LAION-400M**. Estamos falando de 400 milhões de vetores.

* **Custo sem `on_disk`:** Você precisaria de um servidor com centenas de Gigabytes de RAM. Isso custaria milhares de dólares por mês.
* **Custo com `on_disk`:** Você consegue rodar essa mesma busca em uma máquina com apenas  GB ou  GB de RAM, pois o grosso dos dados (terabytes) está "descansando" no SSD, que é muito mais barato.

---

### A "Dobradinha" Perfeita: On-Disk + Quantização

O `on_disk` raramente é usado sozinho em larga escala. A estratégia mestre que o Qdrant ensina é:

1. **Vetores Originais:** `on_disk: True` (Ficam no SSD para economizar dinheiro).
2. **Quantização (Resumo):** `always_ram: True` (Uma versão comprimida e minúscula do vetor fica na RAM).

**O fluxo da busca fica assim:**

1. O Qdrant pesquisa rapidamente na RAM usando os resumos (quantizados).
2. Ele seleciona os, por exemplo, 100 melhores candidatos.
3. **Só para esses 100**, ele vai no disco (`on_disk`), pega o vetor original perfeito, faz o ajuste fino da precisão (rescoring) e te entrega o resultado.

> **Nota:** Usar `on_disk` aumenta um pouco a **latência** (o tempo de resposta), pois ler do SSD é mais lento que ler da RAM. Porém, para 99% das aplicações, essa diferença de milissegundos é imperceptível diante da economia de custos.